In [32]:
from selenium import webdriver
import time
import os

In [2]:
url = 'http://kto.visitkorea.or.kr/kor/notice/data/statis/profit/notice/inout/popup.kto'

In [7]:
driver = webdriver.Chrome("./chromedriver.exe")
driver.get(url)

In [31]:
category = driver.find_element_by_xpath('//*[@id="gubun_2"]')
category.click()
purpose = driver.find_element_by_xpath('//*[@id="gubun_2"]/option[2]')
purpose.click()
monthly = driver.find_element_by_xpath('//*[@id="mothRadio01"]')
monthly.click()
for i in range(8):
    driver.find_element_by_xpath('//*[@id="yyyy"]').click()
    year = driver.find_element_by_css_selector('#yyyy > option:nth-child({})'.format(str(i+1)))
    year.click()
    for i in range(12):
        driver.find_element_by_xpath('//*[@id="mm"]').click()
        month = driver.find_element_by_css_selector('#mm > option:nth-child({})'.format(str(i+1)))
        month.click()
        search = driver.find_element_by_xpath('//*[@id="popContents"]/form[1]/fieldset/div/div[2]/a/img')
        search.click()
        time.sleep(5)




In [36]:
os.rename(r'file C:/Users/kihyu/Downloads/file.xls', r'file C:/Users/kihyu/Downloads/kto_01/2020.xls')

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'file C:/Users/kihyu/Downloads/file.xls' -> 'file C:/Users/kihyu/Downloads/kto_01/2020.xls'